In [3]:
import sys  
sys.path.insert(0, '../../')
import RadonDF_Handler
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [4]:
data = sm.datasets.co2.load_pandas()
y = data.data

In [6]:
y

,co2
1958-03-29,316.1
1958-04-05,317.3
1958-04-12,317.6
1958-04-19,317.5
1958-04-26,316.4
...,...
2001-12-01,370.3
2001-12-08,370.8
2001-12-15,371.2
2001-12-22,371.3


In [9]:
D003_df = pd.read_csv('./../../../../../Data/SensorsData/interpolated_D003_data.csv')
D003_df = D003_df.iloc[913:]
D003_df['time'] =  pd.to_datetime(D003_df['time'], format='%Y-%m-%d %H:%M:%S')
D003_df.index = D003_df['time']
D003_df = D003_df.drop(['time','T','H','P','CO2'],axis = 1)
D003_df 

,Rn
time,
2019-09-19 14:00:00,449.698000
2019-09-19 15:00:00,173.120000
2019-09-19 16:00:00,43.223333
2019-09-19 17:00:00,43.876000
2019-09-19 18:00:00,122.895000
...,...
2021-12-31 19:00:00,956.366667
2021-12-31 20:00:00,1275.534000
2021-12-31 21:00:00,1321.585000


In [1]:
def plotter(df,x,y,title):
    fig = px.line(df, x=x, y=y, title=title)

    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(step="all")
            ])
        )
    )
    fig.show()

In [2]:
# p-value <= 0.05 means data is stationary
def adfullerTest(df):
        result = adfuller(df)
        print('ADF Statistic: %f' % result[0])
        print('p-value: %f' % result[1])
        print('Critical Values:')
        for key, value in result[4].items():
            print('\t%s: %.3f' % (key, value))

        print("-----------------------------------")

In [7]:
# columns = [np.log(D003_df['T'].values),np.log(D003_df['Rn'].values),np.log(D003_df['H'].values),np.log(D003_df['P'].values),np.log(D003_df['CO2'].values)]
adfullerTest(D003_df['Rn'])


ADF Statistic: -6.791733
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
-----------------------------------


In [5]:
# The correlation heatmap is based on the Pearson Correlation coefficient,
# which states that any value lower than 0.8 or greater than -0.8 is insignificant,
# as it is the most commonly used coefficient in statistics.
D003_df.corr()

,T,Rn,H,P,CO2
T,1.000000,-0.464715,-0.058313,-0.149509,0.091459
Rn,-0.464715,1.000000,0.218187,0.044456,-0.174839
H,-0.058313,0.218187,1.000000,-0.146443,0.088599
P,-0.149509,0.044456,-0.146443,1.000000,0.182874
CO2,0.091459,-0.174839,0.088599,0.182874,1.000000


In [ ]:
# run ADF test
from statsmodels.tsa.stattools import adfuller

def adf_test(series,title=''):
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') 
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels) 

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())          
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")
        
# use ADF function       
adf_test(df_H['Rn'])

In [19]:
training_data, testing_data = train_test_split(D003_df, test_size=0.2, shuffle=False)

In [56]:
#training_data = D003_df.sample(frac=0.8,random_state=None)
# testing_data = D003_df.drop(training_data.index)
training_data, testing_data = train_test_split(D003_df, test_size=0.2, shuffle=False)